In [ ]:
#librerías
import geopandas as gpd
import pandas as pd
import numpy as np
import sys
sys.path.append(r"C:\Users\mfpen\Documents\Repositorios\geoloc2\preparacion_datos\src")
from preparacion_inter_puntos import prep
sys.path.append(r"C:\Users\mfpen\Documents\Repositorios\geoloc2\preparacion_datos")
from funciones_cruces_limpieza import *

In [ ]:
#Rutas de bases
curt_path = r"H:\.shortcut-targets-by-id\1xy7wVmj8pxJHqeyV9CG1vOBQs3emChdq\Geo\Data\bases_shp_INEGI\CURT\060 Ixtapan de la Sal\predios_060.shp"
manzanas_path=r"H:\.shortcut-targets-by-id\1xy7wVmj8pxJHqeyV9CG1vOBQs3emChdq\Geo\Data\bases_shp_INEGI\INFORMACION PARA INTELIGENCIA FISCAL\IXTAPAN DE LA SAL\SHAPE\MANZANAS\060 Ixtapan de la Sal\2023 060 MZ.shp"
predialOD_path = r"D:\Secretaria\cruces_bases\Ixtapan\Insumos\ANEXO 3 PRED DICIEMBRE 2022 - Arturo Baez.xlsx"
predial_path = r"D:\Secretaria\cruces_bases\Ixtapan\Insumos\Ixtapan de La Sal_PREDIAL.xlsx"
deudores_path = r"D:\Secretaria\cruces_bases\Ixtapan\Insumos\Ixtapan de La Sal_PREDIALADEUDO.xlsx"
propietarios_path=r"D:\Secretaria\cruces_bases\Ixtapan\Insumos\Ixtapan de La Sal_PROPIETARIO.xlsx"

Lectura de las bases

In [ ]:
#base de curt y manzanas municipales
CURT_mun = gpd.read_file(curt_path)
manzanas_mun = gpd.read_file(manzanas_path)
#Base predial de OneDrive
base_predial_OD = pd.read_excel(predialOD_path)
#Bases de predial
pred_pred = pd.read_excel(predial_path)
pred_prop = pd.read_excel(propietarios_path)
pred_deu = pd.read_excel(deudores_path)

Cruce 1: Base CURT x Base predial de OneDrive

In [ ]:
#Homologar nombres de Clave Catastral x base para los cruces
CURT_mun.rename(columns={'id_cat':'CLAVECATASTRAL'},inplace=True)
manzanas_mun.rename(columns={'cve_cat':'CLAVECATASTRAL'},inplace=True)

In [ ]:
#revisar si hay vacíos en clave catastral
CURT_mun[CURT_mun.CLAVECATASTRAL.isna()]

In [ ]:
base_predial_OD.shape

In [ ]:
base_predial_OD=base_predial_OD.reset_index(drop=True)
base_predial_OD.shape

In [ ]:
base_predial_OD.columns

In [ ]:
#Se puede utilizar esta celda en caso de que la CC no este homologada a CLAVECATASTRAL, SIEMPRE verificar que quede como CLAVECATASTRAL o
#no funcionará para los cruces siguientes
base_predial_OD.rename(columns={'cve_catastral': 'CLAVECATASTRAL'}, inplace=True)
base_predial_OD.shape

In [ ]:
#Se crea una base con los registros que SI tienen CC en predial
base_predial=base_predial_OD[base_predial_OD.CLAVECATASTRAL.notna()]
base_predial.shape

In [ ]:
#quitar duplicados
base_predial.drop_duplicates(subset='CLAVECATASTRAL', inplace=True)
base_predial.shape

In [ ]:
#Se crean la CC a nivel manzana y a nivel predio de la base predial 
base_predial['CLAVE_MANZ']=base_predial.CLAVECATASTRAL.astype(str).str[0:8]+"00"
base_predial['CLAVE_PREDIO']=base_predial.CLAVECATASTRAL.astype(str).str[0:10]
base_predial['CLAVE_CATASTRAL']=base_predial.CLAVECATASTRAL.astype(str).str[0:10]
base_predial.tail(3)

In [ ]:
base_predial.shape

In [ ]:
#Se usa esta función por si hay alguna columna con Nan o 0 o vacías y no llenarnos de columnas
base_predial=delete_empty(base_predial)
base_predial.shape

In [ ]:
len(base_predial.CLAVECATASTRAL.unique())

In [ ]:
base_predial.columns[base_predial.columns.isin(CURT_mun.columns)]
falta= base_predial.columns[~base_predial.columns.isin(CURT_mun.columns)]
falta

In [ ]:
cruce_predial = pd.merge(base_predial[['No. CONSECUTIVO', 'DOMICILIO', 'NOMBRE DEL PROPIETARIO', 'RFC',
       'ESTATUS DE LA CLAVE', 'VALOR CATASTRAL', 'SUPERFICIE DE TERRENO',
       'USO DE ÁREA HOMOGÉNEA', 'PERIODO DE RECAUDACION CORRIENTE REZAGO',
       'PERIODO DE RECAUDACION CORRIENTE', 'EJERCICIO FISCAL', 'CLAVE_MANZ',
       'CLAVE_PREDIO', 'CLAVE_CATASTRAL','CLAVECATASTRAL']],CURT_mun, on="CLAVECATASTRAL", how='inner')
cruce_predial.shape

In [ ]:
#Revisar valores unicos x CC
len(cruce_predial.CLAVECATASTRAL.unique())

In [ ]:
cruce_predial.columns

In [ ]:
#Se crea base de todos los que no cruzaron x CURT de base predial
base_pred_n = base_predial.loc[~base_predial['CLAVECATASTRAL'].isin(cruce_predial['CLAVECATASTRAL'])]
base_pred_n.shape

In [ ]:
cruce_predial.shape, base_pred_n.shape

Cruce 2: Cruce de base predial a nivel predio x base CURT

In [ ]:
#Se utiliza esta función para separar CC en CC manzana y CC predio
separar_digitos(CURT_mun)
CURT_mun

In [ ]:
#Disolución de las geometrías a partir de clavecatastral 
CURT_mun=CURT_mun.dissolve(by='CLAVE_CATASTRAL').reset_index()
CURT_mun.tail(2)

In [ ]:
#Revisar que no haya vacíos
CURT_mun[CURT_mun.CLAVECATASTRAL.isna()]

In [ ]:
base_pred_n.columns[base_pred_n.columns.isin(CURT_mun.columns)]
falta= base_pred_n.columns[~base_pred_n.columns.isin(CURT_mun.columns)]
falta

In [ ]:
cruce_cat=pd.merge(base_pred_n[['No. CONSECUTIVO', 'DOMICILIO', 'NOMBRE DEL PROPIETARIO', 'RFC',
       'ESTATUS DE LA CLAVE', 'VALOR CATASTRAL', 'SUPERFICIE DE TERRENO',
       'USO DE ÁREA HOMOGÉNEA', 'PERIODO DE RECAUDACION CORRIENTE REZAGO',
       'PERIODO DE RECAUDACION CORRIENTE', 'EJERCICIO FISCAL', 'CLAVE_MANZ',
       'CLAVE_PREDIO',
       'CLAVE_CATASTRAL','CLAVECATASTRAL']],CURT_mun, left_on="CLAVE_PREDIO", right_on="CLAVE_CATASTRAL", how='inner')
cruce_cat.shape

In [ ]:
#Revisión de columnas por si hay duplicadas (_x y _y)
cruce_cat.columns

In [ ]:
#Revisar si coinciden las claves cortadas con la completa y así decidir a cuál darle prioridad
cruce_cat[['CLAVE_CATASTRAL_x','CLAVECATASTRAL_x','CLAVE_CATASTRAL_y','CLAVECATASTRAL_y']]

In [ ]:
cruce_cat['CLAVE_CATASTRAL']=cruce_cat.CLAVE_CATASTRAL_x.combine_first(cruce_cat.CLAVE_CATASTRAL_y)
# cruce_cat[cruce_cat.CLAVECATASTRAL.isna()]
#Eliminar columna de CLAVECATASTRAL_y y renombrar columna de CLAVECATASTRAL_x
cruce_cat.drop(['CLAVE_CATASTRAL_y','CLAVE_CATASTRAL_x'],axis=1,inplace=True )
#cruce_cat.rename(columns={'CLAVECATASTRAL_x':'CLAVECATASTRAL'},inplace=True)
cruce_cat.columns

In [ ]:
cruce_cat['CLAVECATASTRAL']=cruce_cat.CLAVECATASTRAL_x.combine_first(cruce_cat.CLAVECATASTRAL_y)
# cruce_cat[cruce_cat.CLAVECATASTRAL.isna()]
#Eliminar columna de CLAVECATASTRAL_y y renombrar columna de CLAVECATASTRAL_x
cruce_cat.drop(['CLAVECATASTRAL_y','CLAVECATASTRAL_x'],axis=1,inplace=True )
#cruce_cat.rename(columns={'CLAVECATASTRAL_x':'CLAVECATASTRAL'},inplace=True)
cruce_cat.columns

In [ ]:
len(cruce_cat.CLAVECATASTRAL.unique())

In [ ]:
#Se vuelve a revisar que coincidan las claves a nivel predio
cruce_cat[['CLAVE_CATASTRAL','CLAVECATASTRAL']]

Cruce 3 : Cruce de base predial a nivel manzana vs (CURT x Manzanas)

In [ ]:
#Se crea base de todos los que no cruzaron x CURT de base predial
base_predial_n = base_pred_n.loc[~base_pred_n['CLAVE_PREDIO'].isin(cruce_cat['CLAVE_PREDIO'])]
base_predial_n.shape

In [ ]:
#Revisar que no haya vacíos
manzanas_mun[manzanas_mun.CLAVECATASTRAL.isna()]

In [ ]:
#Se revisa el CRS de cada base para poder hacer el cruce entre ellas
print("crs CURT: ",CURT_mun.crs)
print("crs Manzanas: ",manzanas_mun.crs)

In [ ]:
#Se le asigna la misma geometría de CURT a la base de manzanas
manzanas_mun=manzanas_mun.to_crs(4326)
# Se hace un overlay para la unión de las bases 
CURT_Man=CURT_mun.overlay(manzanas_mun, how='union',keep_geom_type=False)
CURT_Man.tail(5)

In [ ]:
CURT_Man['CLAVECATASTRAL_1']=CURT_Man.CLAVECATASTRAL_1.combine_first(CURT_Man.CLAVECATASTRAL_2)
CURT_Man

In [ ]:
#Se utiliza esta función para renombrar columna con _1 y eliminar la otra
combine_first_drop(CURT_Man)
CURT_Man

In [ ]:
#Se revisa que coincidan las claves y si hay vacías
CURT_Man[['CLAVECATASTRAL','CLAVE_CATASTRAL']]

In [ ]:
#Se vuelve a generar la clave a nivel manzana x las vacías 
separar_digitos(CURT_Man)
CURT_Man

In [ ]:
#Se vuelve a revisar que coincidan las claves a nivel manzana
CURT_Man[['CLAVECATASTRAL','CLAVE_CATASTRAL']]

In [ ]:
CURT_Man = CURT_Man.loc[~CURT_Man['manz'].astype(str).str.endswith('000')]
CURT_Man.shape

In [ ]:
#Se revisan columnas 
base_predial_n.columns[base_predial_n.columns.isin(manzanas_mun.columns)]
falta= base_predial_n.columns[~base_predial_n.columns.isin(manzanas_mun.columns)]
falta

In [ ]:
cruce_manzana = pd.merge(base_predial_n[['No. CONSECUTIVO', 'DOMICILIO', 'NOMBRE DEL PROPIETARIO', 'RFC',
       'ESTATUS DE LA CLAVE', 'VALOR CATASTRAL', 'SUPERFICIE DE TERRENO',
       'USO DE ÁREA HOMOGÉNEA', 'PERIODO DE RECAUDACION CORRIENTE REZAGO',
       'PERIODO DE RECAUDACION CORRIENTE', 'EJERCICIO FISCAL', 'CLAVE_MANZ',
       'CLAVE_PREDIO', 'CLAVE_CATASTRAL','CLAVECATASTRAL']], CURT_Man, left_on='CLAVE_MANZ', right_on='CLAVE_CATASTRAL', how='inner')
cruce_manzana

In [ ]:
cruce_manzana.columns

In [ ]:
#Se vuelve a revisar que coincidan las claves para decidir a cuál darle prioridad
cruce_manzana[['CLAVECATASTRAL_x','CLAVECATASTRAL_y','CLAVE_CATASTRAL_x','CLAVE_CATASTRAL_y']]

In [ ]:
cruce_manzana['CLAVECATASTRAL']=cruce_manzana.CLAVECATASTRAL_x.combine_first(cruce_manzana.CLAVECATASTRAL_y)
cruce_manzana[cruce_manzana.CLAVECATASTRAL.isna()]
#Eliminar columna de CLAVECATASTRAL_y y renombrar columna de CLAVECATASTRAL_x
cruce_manzana.drop(['CLAVECATASTRAL_y','CLAVECATASTRAL_x'],axis=1,inplace=True )
cruce_manzana.columns

In [ ]:
cruce_manzana['CLAVE_CATASTRAL']=cruce_manzana.CLAVE_CATASTRAL_x.combine_first(cruce_manzana.CLAVE_CATASTRAL_y)
# cruce_manzana[cruce_manzana.CLAVE_CATASTRAL.isna()]
#Eliminar columna de CLAVECATASTRAL_y y renombrar columna de CLAVECATASTRAL_x
cruce_manzana.drop(['CLAVE_CATASTRAL_y','CLAVE_CATASTRAL_x'],axis=1,inplace=True )
cruce_manzana.columns

In [ ]:
cruce_manzana[['CLAVECATASTRAL','CLAVE_CATASTRAL']]

In [ ]:
len(cruce_manzana.CLAVECATASTRAL.unique())

In [ ]:
#Base de los registros que no cruzaron x curt a nivel predio ni manzana
base_predial_nm =base_predial_n.loc[~base_predial_n['CLAVE_MANZ'].isin(cruce_manzana['CLAVE_MANZ'])]
base_predial_nm

In [ ]:
#Se concatenan el cruce 2 con el cruce 3
cruce_predial_manz = pd.concat([cruce_cat,cruce_manzana])
cruce_predial_manz

In [ ]:
cruce_predial_manz[['CLAVECATASTRAL','CLAVE_CATASTRAL']]

In [ ]:
cruce_predial_manz.columns

In [ ]:
#Se concatena el Cruce 1 con el concatenado del Cruce 2 y 3
cruce_predial_completo = pd.concat([cruce_predial,cruce_predial_manz])
cruce_predial_completo

In [ ]:
cruce_predial_completo.CLAVECATASTRAL.value_counts()

In [ ]:
len(cruce_predial_completo.CLAVECATASTRAL.unique())

In [ ]:
cruce_predial_completo.columns

In [ ]:
#Guardar base de predial vs curt
cruce_predial_completo.to_csv(r"I:\Mi unidad\Cruces_predial_otros\Ixtapan\Bases_salida_predial\NUEVO/Ixtapan_cruce_predialxcurt_OD.csv")

Cruce de predial OD con CURT vs Predial(Predial, deudores y propietarios)

In [ ]:
#Base predial
pred_pred.tail(3)

In [ ]:
#Base propietarios
pred_prop.tail(3)

In [ ]:
#Base deudores
pred_deu.tail(3)

In [ ]:
#borrar columnas vacias
delete_empty(pred_prop)
delete_empty(pred_pred)
delete_empty(pred_deu)

In [ ]:
#borrar duplicados de las bases x CLAVECATASTRAL
pred_pred.drop_duplicates(subset='CLAVECATASTRAL', inplace=True)
pred_prop.drop_duplicates(subset='CLAVECATASTRAL', inplace=True)

In [ ]:
#ASEGURARSE QUE EXISTA LA COLUMNA DE EJERCICIO, O EJERCICIO_FISCAL, Y QUE SE LLAME EJERCICIO FISCAL
pred_deu.columns

In [ ]:
#Correr celda si no se llama como debe la columna correspondiente al ejercicio fiscal
pred_deu.rename(columns={'EJERCICIO':'EJERCICIO FISCAL'},inplace=True)

In [ ]:
#Se borran duplicados de la base deudores x CC y después se ordenan x Ejercicio Fiscal

pred_deu.drop_duplicates(subset='CLAVECATASTRAL',keep='first',inplace=True)
pred_deu.sort_values(by='EJERCICIO FISCAL',ascending=False,inplace=True)

In [ ]:
#Se crea la CC a nivel predio de cada base
pred_pred['CLAVE_CATASTRAL']= pred_pred.CLAVECATASTRAL.astype(str).str[0:10]
pred_prop['CLAVE_CATASTRAL']= pred_prop.CLAVECATASTRAL.astype(str).str[0:10]
pred_deu['CLAVE_CATASTRAL']= pred_deu.CLAVECATASTRAL.astype(str).str[0:10]

In [ ]:
#Asegurarse que se cortaron todas las claves y no hay nan (Descomentar una x una)
pred_pred[pred_pred.CLAVE_CATASTRAL.isna()]
# pred_prop[pred_prop.CLAVE_CATASTRAL.isna()]
# pred_deu[pred_deu.CLAVE_CATASTRAL.isna()]

In [ ]:
#Qué columnas están dentro de mi base x cruzar, que no están dentro de la 1ra base 
pred_pred.columns[pred_pred.columns.isin(pred_prop.columns)]
falta= pred_pred.columns[~pred_pred.columns.isin(pred_prop.columns)]
falta

Cruce 4: Predial - Predial vs Predial - Propietarios

In [ ]:
#Cruce de predial general con propietario (el resultado de este cruce es la primera parte de la base completa de predial, no debe tener duplicados por clave catastral unica a 16 digitos y esa es la key de cruce)
predial_general = pd.merge(pred_pred[['CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA','CLAVE_CATASTRAL','CLAVECATASTRAL']], pred_prop, on="CLAVECATASTRAL", how="inner")
predial_general

In [ ]:
#Al cruzar se generara x y y al existir la columna simultaneamente en las bases, se dejará la que tenga mayor peso
predial_general['CLAVE_CATASTRAL']=predial_general.CLAVE_CATASTRAL_x.combine_first(predial_general.CLAVE_CATASTRAL_y)
#ya que se tiene la columna correcta, se eliminan las generadas como x y Y
columns_to_drop = ['CLAVE_CATASTRAL_x', 'CLAVE_CATASTRAL_y']
predial_general = predial_general.drop(columns_to_drop, axis=1)
predial_general


In [ ]:
#Revisar q no haya vacíos
predial_general[predial_general.CLAVE_CATASTRAL.isna()]
predial_general[predial_general.CLAVECATASTRAL.isna()]

In [ ]:
len(predial_general.CLAVECATASTRAL.unique())

In [ ]:
#revisión de columnas entre predial general (cruce 1) y predial adeudos
predial_general.columns[predial_general.columns.isin(pred_deu.columns)]
falta_deu= predial_general.columns[~predial_general.columns.isin(pred_deu.columns)]
falta_deu

Cruce 5: Predial general vs Predial - Deudores

In [ ]:
cruce_pred1=pd.merge(predial_general[['CLAVECATASTRAL','CLAVE_CATASTRAL','CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'NOMBRE', 'APATERNO', 'AMATERNO', 'RFC',
       'CLAVECP', 'CLAVEMUNICIPIO', 'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS',
       'CLAVEASENTAMIENTO', 'EMAIL', 'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR']],pred_deu, on="CLAVECATASTRAL",  how='outer', indicator=True)
cruce_pred1

In [ ]:
#Al cruzar se generara x y y al existir la columna simultaneamente en las bases, se dejará la que tenga mayor peso
cruce_pred1['CLAVE_CATASTRAL']=cruce_pred1.CLAVE_CATASTRAL_x.combine_first(cruce_pred1.CLAVE_CATASTRAL_y)
#ya que se tiene la columna correcta, se eliminan las generadas como x y Y
columns_to_drop = ['CLAVE_CATASTRAL_x', 'CLAVE_CATASTRAL_y']
cruce_pred1 = cruce_pred1.drop(columns_to_drop, axis=1)
cruce_pred1

In [ ]:
cruce_pred1[cruce_pred1.CLAVECATASTRAL.isna()]

In [ ]:
len(cruce_pred1.CLAVECATASTRAL.unique())

In [ ]:
#guardar bases
cruce_pred1.to_csv(r'I:\Mi unidad\Cruces_predial_otros\Ixtapan\Bases_salida_predial\NUEVO/Ixtapan_predial_prop_deu.csv',encoding="utf-8-sig")

In [ ]:
#eliminar la columna de merge en la base a trabajar de cruce_pred1
cruce_pred1=cruce_pred1.drop(['_merge'], axis=1)

In [ ]:
cruce_pred1.columns[cruce_pred1.columns.isin(cruce_predial_completo.columns)]
falta_cruce_pred1= cruce_pred1.columns[~cruce_pred1.columns.isin(cruce_predial_completo.columns)]
falta_cruce_pred1

Cruce 6: Cruce de Predial general + Predial - Deudores vs Predial_OneDrive + CURT

In [ ]:
cruce_CURT_predial = pd.merge(cruce_pred1[['CLAVE_CATASTRAL', 'CLAVECATASTRAL', 'CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'NOMBRE', 'APATERNO', 'AMATERNO',
       'CLAVECP', 'CLAVEMUNICIPIO', 'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS',
       'CLAVEASENTAMIENTO', 'EMAIL', 'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR',
       'CLAVEPERIODOINI', 'CLAVEPERIODOFIN', 'VALORCATASTRAL', 'STATUS']], cruce_predial_completo, on='CLAVE_CATASTRAL', how = 'outer', indicator=True)
cruce_CURT_predial

In [ ]:
#Al cruzar se generara x y y al existir la columna simultaneamente en las bases, se dejará la que tenga mayor peso
cruce_CURT_predial['CLAVECATASTRAL']=cruce_CURT_predial.CLAVECATASTRAL_x.combine_first(cruce_CURT_predial.CLAVECATASTRAL_y)
#ya que se tiene la columna correcta, se eliminan las generadas como x y Y
columns_to_drop = ['CLAVECATASTRAL_x', 'CLAVECATASTRAL_y']
cruce_CURT_predial = cruce_CURT_predial.drop(columns_to_drop, axis=1)
cruce_CURT_predial

In [ ]:
cruce_CURT_predial= cruce_CURT_predial.rename(columns = {'_merge' : 'ID_predial'})

In [ ]:
faltan = base_predial[~base_predial.CLAVECATASTRAL.isin(cruce_CURT_predial.CLAVECATASTRAL)]
faltan.shape

In [ ]:
#Se concatenan los registros de base predial que no cruzaron
base_final_CURT_Predial = pd.concat([cruce_CURT_predial,faltan])
base_final_CURT_Predial

In [ ]:
len(base_final_CURT_Predial.CLAVECATASTRAL.unique())

In [ ]:
#exportar base 
base_final_CURT_Predial.to_csv(r"I:\Mi unidad\Cruces_predial_otros\Ixtapan\Bases_salida_predial\NUEVO/Ixtapan_cruce_curtxpredial_completo.csv",encoding='utf-8-sig',index=False)


In [ ]:
base_final_CURT_Predial = base_final_CURT_Predial.set_geometry('geometry')
base_final_CURT_Predial['geometry']=base_final_CURT_Predial.centroid

In [ ]:
base_final_CURT_Predial['LATITUD']= base_final_CURT_Predial['geometry'].y
base_final_CURT_Predial['LONGITUD']= base_final_CURT_Predial['geometry'].x
base_final_CURT_Predial

In [ ]:
#Revisar si la base tiene LAT y LON
base_final_CURT_Predial[base_final_CURT_Predial.LONGITUD.isna()]


Agregar LAT LON a los registros que no venían en la base de CURT

In [ ]:
base_final_CURT_Predial.loc[base_final_CURT_Predial['LATITUD'].isna(), 'LATITUD'] = 18
base_final_CURT_Predial.loc[base_final_CURT_Predial['LONGITUD'].isna(), 'LONGITUD'] = -99

In [ ]:
base_final_CURT_Predial.drop("geometry",axis=1,inplace=True)

In [ ]:
base_final_CURT_Predial = prep.transform_df_to_gpd(base_final_CURT_Predial, 'LONGITUD','LATITUD', crs= 'EPSG:4326')
base_final_CURT_Predial

In [ ]:
base_final_CURT_Predial[base_final_CURT_Predial.geometry.isna()]

In [ ]:
base_temp=base_final_CURT_Predial.groupby('CLAVE_CATASTRAL').count()[['CLAVECATASTRAL','curt']].reset_index()
base_temp

In [ ]:
base_temp[base_temp.curt!=0]

In [ ]:
base_temp=base_temp.rename(columns={'CLAVECATASTRAL':'ESTIMADO'})
base_temp

In [ ]:
crucexcurt=base_final_CURT_Predial.merge(base_temp[['ESTIMADO','curt','CLAVE_CATASTRAL']],on='CLAVE_CATASTRAL')
crucexcurt

In [ ]:
# crucexcurt.drop("curt_y",axis=1,inplace=True)
crucexcurt.rename(columns={'curt_y':'CURT', "curt_x":"curt"},inplace=True)
crucexcurt

In [ ]:
len(crucexcurt.CLAVECATASTRAL.unique())

In [ ]:
no_predial=base_predial[~(base_predial.CLAVECATASTRAL.isin(crucexcurt['CLAVECATASTRAL']))]
len(no_predial.CLAVECATASTRAL.unique())

In [ ]:
crucexcurt.to_csv(r'I:\Mi unidad\Cruces_predial_otros\Ixtapan\Bases_salida_predial\NUEVO/Ixtapan_Base_predial_PREGEO.csv',encoding="utf-8-sig", index=False)